In [1]:
import os
import numpy as np
from pydub import AudioSegment
import wave
import struct
import pandas as pd
import tensorflow as tf

In [2]:
def load_mp3(filename):
    '''
    输入mp3文件的名，返回特征数据
    '''
    MP3_File = AudioSegment.from_mp3(file=filename)
    MP3_File.export('shiyan.wav', format="wav")#生成wave文件
    #wav文件读取
    '''
    nchannels:声道数,
    sampwidth：采样宽度,
    framerate：帧速率,
    nframes：帧的总量，
    '''
    f = wave.open('shiyan.wav','rb')
    params = f.getparams()
    nchannels, sampwidth, framerate, nframes = params[:4]
    strData = f.readframes(nframes)#读取音频，字符串格式
    waveData = np.frombuffer(strData,dtype=np.int16)#将字符串转化为int
    waveData = waveData*1.0/(max(abs(waveData)))#wave幅值归一化
    temp_result = np.reshape(waveData,[nframes,nchannels])
    f.close()
    nframes=5000000
    result=temp_result[:nframes]
    outData = result#待写入wav的数据，这里仍然取waveData数据
    outData=np.expand_dims(outData,axis=0)
    outData = tf.cast(outData, dtype=tf.float32)
    return outData


In [3]:
def make_songs(result):
    test=np.array(result)
    nframes=5000000
    nchannels=2
    waveData=test[:nframes]
    outData = waveData
    outData = np.reshape(outData,[nframes*nchannels,1])
    outfile = 'outy.wav'
    outwave = wave.open(outfile, 'wb')#定义存储路径以及文件名
    sampwidth = 2
    fs = 44100
    data_size = len(outData)
    framerate = int(fs)
    nframes = data_size
    comptype = "NONE"
    compname = "not compressed"
    outwave.setparams((nchannels, sampwidth, framerate, nframes,
        comptype, compname))
    outData=outData
    for i in range(len(outData)):
        v=outData[i]
        outwave.writeframes(struct.pack('h', int(v * 64000 / 2)))#outData:16位，-32767~32767，注意不要溢出
    outwave.close()

In [4]:
filename = 'wang_final_musicInfo.csv'
frame = pd.DataFrame({'matrix':[],'tag':[]})
data = pd.read_csv(filename)
music_ids = data['歌曲id'].tolist()
for i in range(0,30):
    print(i)
    tmpstr = '王之歌曲存储/'+str(music_ids[i])+'.mp3'
    matrix = load_mp3(tmpstr)
#    matrix1 = np.random.random(size=[1,5000000,2])
    temp_frame = pd.DataFrame({'matrix':[matrix],'tag':[0]})
    frame = pd.concat([frame,temp_frame])
for i in range(5):
    matrix1 = np.random.random(size=[1,5000000,2])
    temp_frame = pd.DataFrame({'matrix':[matrix1],'tag':[1]})
    frame = pd.concat([frame,temp_frame])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [5]:
np.save('frame.npy',frame)
# frame2 = np.loads('frame.npy')